In [ ]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_colwidth = None

In [ ]:
print("Give me the input path")
filename_i = input()
print("Give encoder name")
encoder = input()
make_enc(filename_i,encoder)
print("Give me the query path")
filename_q =input()

make_enc(filename_q,encoder)

Give me the input path
nasdaq_input.csv
Give encoder name
encoder_C.h5
Number of rows and columns: (3651, 100)
[[0.04579599]
 [0.03457418]
 [0.04052691]
 [0.03835925]
 [0.0418942 ]
 [0.0429947 ]
 [0.05301597]
 [0.05861853]
 [0.05850181]
 [0.06403768]]
365
    0        1        2        3     ...     1092     1093     1094     1095
0      a  20.0751  20.5983  28.6123  ...   200.45  152.221  149.331  203.463
1     aa  84.8842  87.1009   125.64  ...  122.889  87.9198  86.9462  115.004
2   aaba     24.7  26.4754  36.6751  ...   166.63  139.835  146.086   211.86
3   aapl  21.8878  21.9443  31.3208  ...  428.019  350.145  356.896  579.277
4    abc  27.3991  27.1462  35.6427  ...  188.178  132.514  129.275  177.702
..   ...      ...      ...      ...  ...      ...      ...      ...      ...
95   cvs  23.2942  23.5781  29.0963  ...  174.409  119.685  119.069  165.257
96   cvx   37.547  38.6107  46.4965  ...  315.433   209.56  209.778  308.415
97     d  33.7286  35.1899  40.9598  ...  211.323  

In [ ]:
  #same for query
def make_enc(filename,encoder):
    df_q =pd.read_csv(filename ,delimiter='\t', header=None)
    df_q = df_q.T
    print(f'Number of rows and columns: {df_q.shape}')
    df_q.head(5)

    to_be_encoded = []
    s=0
    #temp=temp.reshape(-1,1)
    #successfully scaled after dayz xD
    encoded_sc = []
    encoded_set_scaled= []
    to_be_encoded_final = []
    training_set_scaled_cnn= []
    for i in range (0,df_q.shape[1]):
      encoded_set_scaled.append(MinMaxScaler(feature_range = (0, 1)))
      temp=df_q[i][1:]
      temp=np.array(temp)
      temp=temp.reshape(-1,1)
      temp = encoded_set_scaled[i].fit_transform(temp)
      training_set_scaled_cnn.append(temp)  
      s=0
      while(s<len(df_q[0])-1):
        tempo_x_train = temp[s:s+10]
        
        tempo_x_train=np.asarray(tempo_x_train).astype(np.float32)
        to_be_encoded.append(tempo_x_train)
        s=s+10
      to_be_encoded_final.append(np.asarray(to_be_encoded).astype(np.float32))
      to_be_encoded=[]
    print((to_be_encoded_final[0][0]))

    encoder = keras.models.load_model(encoder)
    the_encoded = []
    for i in range(0,df_q.shape[1]):
      the_encoded.append(encoder.predict(to_be_encoded_final[i]))

    print(len(the_encoded[0]))

    new_list_q=[]
    for i in range(0,df_q.shape[1]):
      the_temp_encoded = the_encoded[i].reshape(-1,1)
      the_temp_encoded = encoded_set_scaled[i].inverse_transform(the_temp_encoded)
      k = the_temp_encoded.reshape(the_encoded[0].shape[0]*the_encoded[0].shape[1])
      new_list_q.append(list(k))

    new_df_q = pd.DataFrame()

    for i in range(0,df_q.shape[1]):
      new_list_q[i].insert(0,df_q[i][0])
      new_df_q[i]=new_list_q[i]

    new_df_q = new_df_q.T

    print(new_df_q)
    filname=filename[:-4]
    filename= filename + "enc.csv"
    k = new_df_q.to_csv(filname, sep='\t', encoding='utf-8',header=False,index=False)
